In [0]:
%tensorflow_version 2.x

#Import needed libraries
import tensorflow               as tf
import numpy                    as np
import cv2
import os, datetime
import tensorflow.keras.backend as K
import glob
import math

from skimage.exposure                       import rescale_intensity
from keras.preprocessing.image              import ImageDataGenerator
from tensorflow.keras                       import regularizers
from tensorflow.keras.optimizers            import Adam
from tensorflow.keras.preprocessing.image   import load_img, img_to_array
from sklearn.model_selection                import train_test_split
from keras.utils.generic_utils              import get_custom_objects
from tensorflow.keras.applications          import Xception
from tensorflow.keras.layers                import Dense, Flatten, Dropout, Activation
from tensorflow.keras.models                import Model
from tensorflow.keras.callbacks             import ModelCheckpoint

#making sure that the correct version is selected
print(tf.__version__)

#Load the Xception model and setting some hyperparameters
image_size = 299
batch_size_var = 6
num_epochs = 100
augmentation_factor = 12
Xception_conv_base = Xception(weights= 'imagenet', include_top=True, input_shape=(image_size, image_size, 3))

Using TensorFlow backend.


2.2.0
91889664/91884032 [==============================] - 1s 0us/step


In [0]:
#function for creating datasets
def dataset_creator(path, file_name, img_height, img_width):
    
    image = list ()
    for img_name  in  glob.glob( pathname = path  + '/' + file_name + '/**.png' ):
        img = load_img( path = img_name, color_mode = 'rgb', target_size = (img_height, img_width), interpolation='bicubic')
        img = img_to_array( img = img,  data_format = 'channels_last')
        p2, p98 = np.percentile(img, (2, 98))
        img = rescale_intensity(img, in_range=(p2, p98))
        img = img/255.0
        image.append(img)
    X = np.array(image).astype(dtype = np.float32 )
    Y = (np.genfromtxt(path + '/' +file_name +  '/' + file_name + '.csv', delimiter=',')).astype(dtype = np.float32 )
   
    return  train_test_split(X, Y, test_size = 0.25, random_state = 18 )

In the next cells we create necessary dataset for training our models.

In [0]:
A1_X_train, A1_X_test, A1_Y_train, A1_Y_test = dataset_creator('/content/drive/My Drive/PlantDataset','Ara2012',image_size,image_size)

In [0]:
A2_X_train, A2_X_test, A2_Y_train, A2_Y_test = dataset_creator('/content/drive/My Drive/PlantDataset','Ara2013',image_size,image_size)

In [0]:
A3_X_train, A3_X_test, A3_Y_train, A3_Y_test = dataset_creator('/content/drive/My Drive/PlantDataset','Tobacco',image_size,image_size)

In [0]:
A4_X_train, A4_X_test, A4_Y_train, A4_Y_test = dataset_creator('/content/drive/My Drive/PlantDataset','A4',image_size,image_size)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
A1A2_X_train = np.concatenate([A1_X_train,A2_X_train])  
A1A2_X_test  = np.concatenate([A1_X_test,A2_X_test])
A1A2_Y_train = np.concatenate([A1_Y_train,A2_Y_train])
A1A2_Y_test  = np.concatenate([A1_Y_test,A2_Y_test])

In [0]:
A1A4_X_train = np.concatenate([A1_X_train,A4_X_train])  
A1A4_X_test  = np.concatenate([A1_X_test,A4_X_test])
A1A4_Y_train = np.concatenate([A1_Y_train,A4_Y_train])
A1A4_Y_test  = np.concatenate([A1_Y_test,A4_Y_test])

In [0]:
A2A4_X_train = np.concatenate([A2_X_train,A4_X_train])  
A2A4_X_test  = np.concatenate([A2_X_test,A4_X_test])
A2A4_Y_train = np.concatenate([A2_Y_train,A4_Y_train])
A2A4_Y_test  = np.concatenate([A2_Y_test,A4_Y_test])

In [0]:
Ac_X_train    = np.concatenate([A1_X_train,A2_X_train,A3_X_train,A4_X_train])  
Ac_X_test     = np.concatenate([A1_X_test,A2_X_test,A3_X_test,A4_X_test])
Ac_Y_train    = np.concatenate([A1_Y_train,A2_Y_train,A3_Y_train,A4_Y_train])
Ac_Y_test     = np.concatenate([A1_Y_test,A2_Y_test,A3_Y_test,A4_Y_test])

In [0]:
#print(Ac_X_train.shape)
#print(A5_X_test.shape)
#print(A5_Y_train.shape)
#print(A5_Y_test.shape)

In [0]:
#Defining two metrics

#DiC
def mean_Difference_in_Count(y_true, y_pred):
   return K.cast(K.mean( K.round(y_pred) - y_true), 'float32') 

def std_Difference_in_Count(y_true, y_pred):
   return K.cast(K.std( K.round(y_pred) - y_true), 'float32') 

#|DiC|
def mean_absolute_Difference_in_Count(y_true, y_pred):
  return K.cast(K.mean(K.abs(K.round(y_pred) - y_true)), 'float32') 

def std_absolute_Difference_in_Count(y_true, y_pred):
  return K.cast(K.std(K.abs(K.round(y_pred) - y_true)), 'float32')
  
#MSE
def MSE(y_true, y_pred):
  return K.cast(K.mean(K.pow((K.round(y_pred) - y_true),2)), 'float32')

#last layer activation function
#code inspired from https://stackoverflow.com/questions/43915482/how-do-you-create-a-custom-activation-function-with-keras
def round_relu(x):
    out = K.round(tf.keras.activations.relu(x))
    out = np.array(out)
    return out.astype('float32')
get_custom_objects().update({'round_relu': Activation(round_relu)})

In [0]:
adam_optimizer = Adam(learning_rate=0.001)
#Creating Base Architecture
def create_base_model():
    base_model = tf.keras.models.Sequential([
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(1024, activation='relu',kernel_initializer='glorot_normal'),
             tf.keras.layers.Dropout(0.2),
             tf.keras.layers.Dense(512, activation = 'relu',kernel_initializer='glorot_normal',kernel_regularizer=regularizers.l2(0.01)),
             tf.keras.layers.Dropout(0.2),
             tf.keras.layers.Dense(1, activation = 'relu',kernel_initializer='glorot_normal')
            ])


    base_model.compile(
                adam_optimizer,
                loss='mean_squared_error',
                metrics=[mean_Difference_in_Count,
                         std_Difference_in_Count,
                         mean_absolute_Difference_in_Count,
                         std_absolute_Difference_in_Count,
                         MSE
                         ])
    return base_model
#base_model.build(input_shape=(None,299,299,3))
#base_model.summary()

#Creating Xception architecture and adjusting the last layers to make it compatible with a counting problem
#code inspired from https://github.com/keras-team/keras/issues/3465
def create_Xception_model():
    x = Xception_conv_base.layers[-2].output
    x = Dense(1024, activation = 'relu',kernel_initializer='glorot_normal',)(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation = 'relu',kernel_initializer='glorot_normal',kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.2)(x)
    predictions = Dense(1, activation = 'relu' ,kernel_initializer='glorot_normal')(x)
    Xception_conv_altered= Model(inputs = Xception_conv_base.input, outputs = predictions)

    Xception_conv_altered.compile(
                optimizer=adam_optimizer,
                loss='mean_squared_error', 
                metrics=[mean_Difference_in_Count,
                         std_Difference_in_Count,
                         mean_absolute_Difference_in_Count,
                         std_absolute_Difference_in_Count,
                         MSE])
    
    return Xception_conv_altered
#Xception_conv_altered.summary()

In [0]:
#data augmentation function
def dataset_augmentor(X_train,Y_train):
    datagen_train = ImageDataGenerator(
      zoom_range=0.1,
      rotation_range=170,
      horizontal_flip=True,
      vertical_flip=True)

    return datagen_train.flow(X_train, Y_train, batch_size = batch_size_var)

In [0]:
#Training the models
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
#file_path = '/content/drive/My Drive/model/eps={epoch:03d}_valLoss={val_loss:.4f}.hdf5'
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.1, patience=5 ,restore_best_weights=True)
#checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1,
#                             save_best_only=True, mode='min')

#Base architecture
def fitter_functions (model,X_train,X_test,Y_train,Y_test,augmentation):

  if(augmentation):
    train_set = dataset_augmentor(X_train,Y_train)
    model.fit(
        train_set,
        steps_per_epoch= 12 * math.ceil(len(train_set)),
        epochs=100,
        validation_data = (X_test, Y_test),
        callbacks=[early_stopping])
    
  else:
     model.fit(
        x=X_train, 
        y=Y_train, 
        batch_size= batch_size_var,
        epochs=100,
        validation_data = (X_test, Y_test),
        callbacks=[early_stopping])  


In [0]:
fitter_functions(create_base_model(),A1_X_train,A1_X_test,A1_Y_train,A1_Y_test,False)

Epoch 1/100
15/15 [==============================] - 23s 2s/step - loss: 60.2430 - mean_Difference_in_Count: -2.2111 - std_Difference_in_Count: 2.0287 - mean_absolute_Difference_in_Count: 5.9444 - std_absolute_Difference_in_Count: 1.7706 - MSE: 54.7889 - val_loss: 31.2227 - val_mean_Difference_in_Count: 4.6333 - val_std_Difference_in_Count: 2.1718 - val_mean_absolute_Difference_in_Count: 4.7000 - val_std_absolute_Difference_in_Count: 2.0609 - val_MSE: 27.1000
Epoch 2/100
15/15 [==============================] - 22s 1s/step - loss: 16.6580 - mean_Difference_in_Count: 0.0111 - std_Difference_in_Count: 2.2556 - mean_absolute_Difference_in_Count: 2.9000 - std_absolute_Difference_in_Count: 1.8577 - MSE: 13.0111 - val_loss: 12.9103 - val_mean_Difference_in_Count: 2.3000 - val_std_Difference_in_Count: 1.9928 - val_mean_absolute_Difference_in_Count: 2.7000 - val_std_absolute_Difference_in_Count: 1.4878 - val_MSE: 9.9000
Epoch 3/100
15/15 [==============================] - 22s 1s/step - loss: 1

In [0]:
fitter_functions(create_base_model(),A2_X_train,A2_X_test,A2_Y_train,A2_Y_test,False)

Epoch 1/100
21/21 [==============================] - 33s 2s/step - loss: 28.5451 - mean_Difference_in_Count: -0.6190 - std_Difference_in_Count: 2.0142 - mean_absolute_Difference_in_Count: 3.8254 - std_absolute_Difference_in_Count: 1.6817 - MSE: 24.1905 - val_loss: 6.4060 - val_mean_Difference_in_Count: -0.9286 - val_std_Difference_in_Count: 1.6016 - val_mean_absolute_Difference_in_Count: 1.6905 - val_std_absolute_Difference_in_Count: 1.0019 - val_MSE: 4.5000
Epoch 2/100
21/21 [==============================] - 33s 2s/step - loss: 7.9929 - mean_Difference_in_Count: -0.4206 - std_Difference_in_Count: 2.0854 - mean_absolute_Difference_in_Count: 1.9286 - std_absolute_Difference_in_Count: 1.2698 - MSE: 5.8968 - val_loss: 5.5064 - val_mean_Difference_in_Count: 0.3810 - val_std_Difference_in_Count: 1.5368 - val_mean_absolute_Difference_in_Count: 1.5238 - val_std_absolute_Difference_in_Count: 1.0151 - val_MSE: 3.5238
Epoch 3/100
21/21 [==============================] - 31s 1s/step - loss: 8.63

In [0]:
fitter_functions(create_base_model(),A3_X_train,A3_X_test,A3_Y_train,A3_Y_test,False)

Epoch 1/100
8/8 [==============================] - 12s 1s/step - loss: 23.3357 - mean_Difference_in_Count: -0.9792 - std_Difference_in_Count: 2.3762 - mean_absolute_Difference_in_Count: 3.0833 - std_absolute_Difference_in_Count: 2.0394 - MSE: 16.8750 - val_loss: 19.6201 - val_mean_Difference_in_Count: -1.8611 - val_std_Difference_in_Count: 2.9202 - val_mean_absolute_Difference_in_Count: 2.8611 - val_std_absolute_Difference_in_Count: 2.0182 - val_MSE: 14.4167
Epoch 2/100
8/8 [==============================] - 12s 1s/step - loss: 11.6354 - mean_Difference_in_Count: -0.3646 - std_Difference_in_Count: 2.1477 - mean_absolute_Difference_in_Count: 1.9896 - std_absolute_Difference_in_Count: 1.3407 - MSE: 6.8021 - val_loss: 16.7797 - val_mean_Difference_in_Count: -0.0833 - val_std_Difference_in_Count: 2.9884 - val_mean_absolute_Difference_in_Count: 2.5833 - val_std_absolute_Difference_in_Count: 1.6627 - val_MSE: 11.6389
Epoch 3/100
8/8 [==============================] - 12s 1s/step - loss: 9.87

In [0]:
fitter_functions(create_base_model(),A4_X_train,A4_X_test,A4_Y_train,A4_Y_test,False)

Epoch 1/100
78/78 [==============================] - 4s 46ms/step - loss: 19.1447 - mean_Difference_in_Count: -0.3483 - std_Difference_in_Count: 2.4226 - mean_absolute_Difference_in_Count: 2.8355 - std_absolute_Difference_in_Count: 1.7570 - MSE: 16.0192 - val_loss: 8.1346 - val_mean_Difference_in_Count: 0.8205 - val_std_Difference_in_Count: 1.9838 - val_mean_absolute_Difference_in_Count: 1.9744 - val_std_absolute_Difference_in_Count: 1.3150 - val_MSE: 6.3205
Epoch 2/100
78/78 [==============================] - 3s 40ms/step - loss: 8.8449 - mean_Difference_in_Count: -0.0919 - std_Difference_in_Count: 2.0887 - mean_absolute_Difference_in_Count: 2.0491 - std_absolute_Difference_in_Count: 1.4078 - MSE: 7.2543 - val_loss: 6.3215 - val_mean_Difference_in_Count: 0.6154 - val_std_Difference_in_Count: 1.8461 - val_mean_absolute_Difference_in_Count: 1.7308 - val_std_absolute_Difference_in_Count: 1.2123 - val_MSE: 4.9872
Epoch 3/100
78/78 [==============================] - 3s 36ms/step - loss: 7.

In [0]:
fitter_functions(create_base_model(),Ac_X_train,Ac_X_test,Ac_Y_train,Ac_Y_test,False)

Epoch 1/100
122/122 [==============================] - 5s 45ms/step - loss: 26.8935 - mean_Difference_in_Count: -0.1557 - std_Difference_in_Count: 3.1426 - mean_absolute_Difference_in_Count: 3.5902 - std_absolute_Difference_in_Count: 2.1695 - MSE: 24.6038 - val_loss: 12.3848 - val_mean_Difference_in_Count: -1.4065 - val_std_Difference_in_Count: 2.2709 - val_mean_absolute_Difference_in_Count: 2.5285 - val_std_absolute_Difference_in_Count: 1.7096 - val_MSE: 10.6423
Epoch 2/100
122/122 [==============================] - 4s 37ms/step - loss: 12.0830 - mean_Difference_in_Count: -0.1926 - std_Difference_in_Count: 2.6180 - mean_absolute_Difference_in_Count: 2.5451 - std_absolute_Difference_in_Count: 1.6601 - MSE: 10.4904 - val_loss: 7.7682 - val_mean_Difference_in_Count: 0.2053 - val_std_Difference_in_Count: 2.0762 - val_mean_absolute_Difference_in_Count: 1.8963 - val_std_absolute_Difference_in_Count: 1.3595 - val_MSE: 6.2541
Epoch 3/100
122/122 [==============================] - 4s 34ms/step

In [0]:
fitter_functions(create_Xception_model(),A1_X_train,A1_X_test,A1_Y_train,A1_Y_test,False)

Epoch 1/100
15/15 [==============================] - 132s 9s/step - loss: 82.5234 - mean_Difference_in_Count: -3.9222 - std_Difference_in_Count: 3.1652 - mean_absolute_Difference_in_Count: 7.2333 - std_absolute_Difference_in_Count: 2.3732 - MSE: 77.5667 - val_loss: 170.2195 - val_mean_Difference_in_Count: -12.4667 - val_std_Difference_in_Count: 1.5096 - val_mean_absolute_Difference_in_Count: 12.4667 - val_std_absolute_Difference_in_Count: 1.5096 - val_MSE: 158.1333
Epoch 2/100
15/15 [==============================] - 129s 9s/step - loss: 16.1127 - mean_Difference_in_Count: -0.2444 - std_Difference_in_Count: 2.5045 - mean_absolute_Difference_in_Count: 2.5778 - std_absolute_Difference_in_Count: 1.8543 - MSE: 11.5556 - val_loss: 111.4736 - val_mean_Difference_in_Count: -10.3000 - val_std_Difference_in_Count: 1.3873 - val_mean_absolute_Difference_in_Count: 10.3000 - val_std_absolute_Difference_in_Count: 1.3873 - val_MSE: 108.3667
Epoch 3/100
15/15 [==============================] - 134s 9s

In [0]:
fitter_functions(create_Xception_model(),A2_X_train,A2_X_test,A2_Y_train,A2_Y_test,False)

Epoch 1/100
21/21 [==============================] - 181s 9s/step - loss: 28.3450 - mean_Difference_in_Count: -2.1984 - std_Difference_in_Count: 2.6766 - mean_absolute_Difference_in_Count: 4.0079 - std_absolute_Difference_in_Count: 2.0564 - MSE: 24.1349 - val_loss: 324.9576 - val_mean_Difference_in_Count: 17.8571 - val_std_Difference_in_Count: 1.5740 - val_mean_absolute_Difference_in_Count: 17.8571 - val_std_absolute_Difference_in_Count: 1.5740 - val_MSE: 322.3810
Epoch 2/100
21/21 [==============================] - 184s 9s/step - loss: 19.6454 - mean_Difference_in_Count: -0.8730 - std_Difference_in_Count: 2.9089 - mean_absolute_Difference_in_Count: 3.3492 - std_absolute_Difference_in_Count: 1.9647 - MSE: 16.6349 - val_loss: 87.7896 - val_mean_Difference_in_Count: 9.2381 - val_std_Difference_in_Count: 1.5321 - val_mean_absolute_Difference_in_Count: 9.2381 - val_std_absolute_Difference_in_Count: 1.5321 - val_MSE: 88.8095
Epoch 3/100
21/21 [==============================] - 182s 9s/step 

In [0]:
fitter_functions(create_Xception_model(),A3_X_train,A3_X_test,A3_Y_train,A3_Y_test,False)

Epoch 1/100
8/8 [==============================] - 63s 8s/step - loss: 38.6054 - mean_Difference_in_Count: -5.2187 - std_Difference_in_Count: 1.5744 - mean_absolute_Difference_in_Count: 5.2812 - std_absolute_Difference_in_Count: 1.4874 - MSE: 32.8229 - val_loss: 12.4909 - val_mean_Difference_in_Count: -0.9444 - val_std_Difference_in_Count: 2.7908 - val_mean_absolute_Difference_in_Count: 2.5000 - val_std_absolute_Difference_in_Count: 1.4823 - val_MSE: 9.8333
Epoch 2/100
8/8 [==============================] - 62s 8s/step - loss: 11.3039 - mean_Difference_in_Count: 0.1250 - std_Difference_in_Count: 1.6548 - mean_absolute_Difference_in_Count: 2.3750 - std_absolute_Difference_in_Count: 1.3705 - MSE: 9.7708 - val_loss: 27.6510 - val_mean_Difference_in_Count: -4.5833 - val_std_Difference_in_Count: 2.3629 - val_mean_absolute_Difference_in_Count: 4.5833 - val_std_absolute_Difference_in_Count: 2.3629 - val_MSE: 28.0833
Epoch 3/100
8/8 [==============================] - 62s 8s/step - loss: 7.3606

In [0]:
fitter_functions(create_Xception_model(),A4_X_train,A4_X_test,A4_Y_train,A4_Y_test,False)

Epoch 1/100
78/78 [==============================] - 14s 181ms/step - loss: 19.0917 - mean_Difference_in_Count: -0.8184 - std_Difference_in_Count: 1.7288 - mean_absolute_Difference_in_Count: 2.5278 - std_absolute_Difference_in_Count: 1.4126 - MSE: 17.0064 - val_loss: 6.7592 - val_mean_Difference_in_Count: -0.3141 - val_std_Difference_in_Count: 2.0306 - val_mean_absolute_Difference_in_Count: 1.7500 - val_std_absolute_Difference_in_Count: 1.3529 - val_MSE: 5.4423
Epoch 2/100
78/78 [==============================] - 13s 168ms/step - loss: 4.2899 - mean_Difference_in_Count: -0.1218 - std_Difference_in_Count: 1.3533 - mean_absolute_Difference_in_Count: 1.2799 - std_absolute_Difference_in_Count: 1.0159 - MSE: 3.2329 - val_loss: 5.5817 - val_mean_Difference_in_Count: -0.7051 - val_std_Difference_in_Count: 1.7940 - val_mean_absolute_Difference_in_Count: 1.5513 - val_std_absolute_Difference_in_Count: 1.2975 - val_MSE: 4.6923
Epoch 3/100
78/78 [==============================] - 13s 168ms/step - 

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,Ac_X_test,Ac_Y_train,Ac_Y_test,False)

Epoch 1/100
122/122 [==============================] - 22s 180ms/step - loss: 14.5308 - mean_Difference_in_Count: -0.5041 - std_Difference_in_Count: 1.8059 - mean_absolute_Difference_in_Count: 2.2828 - std_absolute_Difference_in_Count: 1.4393 - MSE: 12.9249 - val_loss: 5.4476 - val_mean_Difference_in_Count: 0.5549 - val_std_Difference_in_Count: 1.7392 - val_mean_absolute_Difference_in_Count: 1.6240 - val_std_absolute_Difference_in_Count: 1.1027 - val_MSE: 4.3760
Epoch 2/100
122/122 [==============================] - 20s 167ms/step - loss: 3.7760 - mean_Difference_in_Count: -0.0683 - std_Difference_in_Count: 1.3013 - mean_absolute_Difference_in_Count: 1.3033 - std_absolute_Difference_in_Count: 0.9580 - MSE: 3.0464 - val_loss: 7.1514 - val_mean_Difference_in_Count: -1.4390 - val_std_Difference_in_Count: 1.7132 - val_mean_absolute_Difference_in_Count: 1.9390 - val_std_absolute_Difference_in_Count: 1.3071 - val_MSE: 6.2358
Epoch 3/100
122/122 [==============================] - 20s 167ms/st

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,A1_X_test,Ac_Y_train,A1_Y_test,False)

Epoch 1/100
122/122 [==============================] - 20s 167ms/step - loss: 13.2440 - mean_Difference_in_Count: -0.4932 - std_Difference_in_Count: 1.5945 - mean_absolute_Difference_in_Count: 2.1626 - std_absolute_Difference_in_Count: 1.2925 - MSE: 11.5861 - val_loss: 6.9636 - val_mean_Difference_in_Count: -1.9333 - val_std_Difference_in_Count: 1.3078 - val_mean_absolute_Difference_in_Count: 1.9333 - val_std_absolute_Difference_in_Count: 1.3078 - val_MSE: 5.9333
Epoch 2/100
122/122 [==============================] - 19s 158ms/step - loss: 3.0264 - mean_Difference_in_Count: -0.0478 - std_Difference_in_Count: 1.2014 - mean_absolute_Difference_in_Count: 1.0615 - std_absolute_Difference_in_Count: 0.8530 - MSE: 2.1762 - val_loss: 6.6516 - val_mean_Difference_in_Count: -2.0000 - val_std_Difference_in_Count: 1.1968 - val_mean_absolute_Difference_in_Count: 2.0000 - val_std_absolute_Difference_in_Count: 1.1968 - val_MSE: 5.8000
Epoch 3/100
122/122 [==============================] - 19s 159ms/s

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,A2_X_test,Ac_Y_train,A2_Y_test,False)

Epoch 1/100
122/122 [==============================] - 20s 168ms/step - loss: 11.5341 - mean_Difference_in_Count: -0.5232 - std_Difference_in_Count: 1.1270 - mean_absolute_Difference_in_Count: 1.8511 - std_absolute_Difference_in_Count: 0.9820 - MSE: 9.8593 - val_loss: 5.4273 - val_mean_Difference_in_Count: -0.0952 - val_std_Difference_in_Count: 1.7127 - val_mean_absolute_Difference_in_Count: 1.6667 - val_std_absolute_Difference_in_Count: 1.0852 - val_MSE: 4.6190
Epoch 2/100
122/122 [==============================] - 19s 158ms/step - loss: 2.3080 - mean_Difference_in_Count: -0.0355 - std_Difference_in_Count: 0.9240 - mean_absolute_Difference_in_Count: 0.8443 - std_absolute_Difference_in_Count: 0.7362 - MSE: 1.5000 - val_loss: 5.0437 - val_mean_Difference_in_Count: -0.3810 - val_std_Difference_in_Count: 1.5959 - val_mean_absolute_Difference_in_Count: 1.6190 - val_std_absolute_Difference_in_Count: 1.0091 - val_MSE: 4.1905
Epoch 3/100
122/122 [==============================] - 19s 157ms/st

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,A3_X_test,Ac_Y_train,A3_Y_test,False)

Epoch 1/100
122/122 [==============================] - 20s 166ms/step - loss: 12.0710 - mean_Difference_in_Count: -0.5656 - std_Difference_in_Count: 1.1978 - mean_absolute_Difference_in_Count: 1.7650 - std_absolute_Difference_in_Count: 0.9771 - MSE: 10.4180 - val_loss: 9.5314 - val_mean_Difference_in_Count: 0.4167 - val_std_Difference_in_Count: 2.4498 - val_mean_absolute_Difference_in_Count: 1.9167 - val_std_absolute_Difference_in_Count: 1.6037 - val_MSE: 7.4167
Epoch 2/100
122/122 [==============================] - 19s 158ms/step - loss: 2.5193 - mean_Difference_in_Count: 0.0246 - std_Difference_in_Count: 0.9834 - mean_absolute_Difference_in_Count: 0.9481 - std_absolute_Difference_in_Count: 0.7856 - MSE: 2.0082 - val_loss: 8.6414 - val_mean_Difference_in_Count: -0.0556 - val_std_Difference_in_Count: 2.2848 - val_mean_absolute_Difference_in_Count: 1.7778 - val_std_absolute_Difference_in_Count: 1.4422 - val_MSE: 7.0000
Epoch 3/100
122/122 [==============================] - 19s 157ms/ste

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,A4_X_test,Ac_Y_train,A4_Y_test,False)

Epoch 1/100
122/122 [==============================] - 21s 172ms/step - loss: 14.6749 - mean_Difference_in_Count: -0.5833 - std_Difference_in_Count: 1.3408 - mean_absolute_Difference_in_Count: 1.9631 - std_absolute_Difference_in_Count: 1.0843 - MSE: 12.8320 - val_loss: 5.2283 - val_mean_Difference_in_Count: -0.9038 - val_std_Difference_in_Count: 1.5794 - val_mean_absolute_Difference_in_Count: 1.4679 - val_std_absolute_Difference_in_Count: 1.1653 - val_MSE: 4.0962
Epoch 2/100
122/122 [==============================] - 20s 165ms/step - loss: 2.5594 - mean_Difference_in_Count: -0.0383 - std_Difference_in_Count: 1.0186 - mean_absolute_Difference_in_Count: 0.9016 - std_absolute_Difference_in_Count: 0.7814 - MSE: 1.6940 - val_loss: 4.1488 - val_mean_Difference_in_Count: -0.3013 - val_std_Difference_in_Count: 1.6755 - val_mean_absolute_Difference_in_Count: 1.4423 - val_std_absolute_Difference_in_Count: 1.0303 - val_MSE: 3.5064
Epoch 3/100
122/122 [==============================] - 20s 164ms/s

In [0]:
fitter_functions(create_Xception_model(),A1_X_train,A1_X_test,A1_Y_train,A1_Y_test,True)

Epoch 1/100
180/180 [==============================] - 30s 168ms/step - loss: 16.1058 - mean_Difference_in_Count: -0.6963 - std_Difference_in_Count: 1.5929 - mean_absolute_Difference_in_Count: 2.3519 - std_absolute_Difference_in_Count: 1.1336 - MSE: 15.0907 - val_loss: 3.6847 - val_mean_Difference_in_Count: -0.5000 - val_std_Difference_in_Count: 1.5864 - val_mean_absolute_Difference_in_Count: 1.3000 - val_std_absolute_Difference_in_Count: 1.0376 - val_MSE: 2.7667
Epoch 2/100
180/180 [==============================] - 29s 160ms/step - loss: 3.4883 - mean_Difference_in_Count: -0.0546 - std_Difference_in_Count: 1.3971 - mean_absolute_Difference_in_Count: 1.3565 - std_absolute_Difference_in_Count: 0.9255 - MSE: 3.0046 - val_loss: 3.1099 - val_mean_Difference_in_Count: -0.4667 - val_std_Difference_in_Count: 1.5861 - val_mean_absolute_Difference_in_Count: 1.2667 - val_std_absolute_Difference_in_Count: 1.0625 - val_MSE: 2.7333
Epoch 3/100
180/180 [==============================] - 29s 159ms/s

In [0]:
fitter_functions(create_Xception_model(),A2_X_train,A2_X_test,A2_Y_train,A2_Y_test,True)

Epoch 1/100
252/252 [==============================] - 42s 165ms/step - loss: 7.0875 - mean_Difference_in_Count: -0.3042 - std_Difference_in_Count: 1.6517 - mean_absolute_Difference_in_Count: 1.8148 - std_absolute_Difference_in_Count: 1.0918 - MSE: 6.4273 - val_loss: 4.1443 - val_mean_Difference_in_Count: -0.7063 - val_std_Difference_in_Count: 1.8157 - val_mean_absolute_Difference_in_Count: 1.5312 - val_std_absolute_Difference_in_Count: 1.2789 - val_MSE: 3.9813
Epoch 2/100
252/252 [==============================] - 40s 159ms/step - loss: 3.6788 - mean_Difference_in_Count: -0.0913 - std_Difference_in_Count: 1.4504 - mean_absolute_Difference_in_Count: 1.4511 - std_absolute_Difference_in_Count: 0.9794 - MSE: 3.4947 - val_loss: 4.6878 - val_mean_Difference_in_Count: -0.0531 - val_std_Difference_in_Count: 2.0646 - val_mean_absolute_Difference_in_Count: 1.8406 - val_std_absolute_Difference_in_Count: 1.1367 - val_MSE: 4.6906
Epoch 3/100
252/252 [==============================] - 40s 159ms/ste

In [0]:
fitter_functions(create_Xception_model(),A3_X_train,A3_X_test,A3_Y_train,A3_Y_test,True)

Epoch 1/100
96/96 [==============================] - 16s 168ms/step - loss: 6.3137 - mean_Difference_in_Count: -0.2509 - std_Difference_in_Count: 1.4285 - mean_absolute_Difference_in_Count: 1.6554 - std_absolute_Difference_in_Count: 1.0295 - MSE: 5.1484 - val_loss: 10.9506 - val_mean_Difference_in_Count: -1.1250 - val_std_Difference_in_Count: 3.0797 - val_mean_absolute_Difference_in_Count: 2.3750 - val_std_absolute_Difference_in_Count: 2.2604 - val_MSE: 10.7500
Epoch 2/100
96/96 [==============================] - 15s 158ms/step - loss: 2.6220 - mean_Difference_in_Count: -0.0781 - std_Difference_in_Count: 1.2929 - mean_absolute_Difference_in_Count: 1.1753 - std_absolute_Difference_in_Count: 0.8713 - MSE: 2.4479 - val_loss: 15.0102 - val_mean_Difference_in_Count: 0.8750 - val_std_Difference_in_Count: 3.6891 - val_mean_absolute_Difference_in_Count: 3.1250 - val_std_absolute_Difference_in_Count: 2.1469 - val_MSE: 14.3750
Epoch 3/100
96/96 [==============================] - 15s 157ms/step -

In [0]:
fitter_functions(create_Xception_model(),A4_X_train,A4_X_test,A4_Y_train,A4_Y_test,True)

Epoch 1/100
936/936 [==============================] - 152s 162ms/step - loss: 5.8453 - mean_Difference_in_Count: -0.1494 - std_Difference_in_Count: 1.5150 - mean_absolute_Difference_in_Count: 1.5597 - std_absolute_Difference_in_Count: 1.1024 - MSE: 5.2420 - val_loss: 4.2955 - val_mean_Difference_in_Count: -0.8071 - val_std_Difference_in_Count: 1.8398 - val_mean_absolute_Difference_in_Count: 1.4625 - val_std_absolute_Difference_in_Count: 1.3783 - val_MSE: 4.0696
Epoch 2/100
936/936 [==============================] - 150s 160ms/step - loss: 3.0538 - mean_Difference_in_Count: -0.0614 - std_Difference_in_Count: 1.3317 - mean_absolute_Difference_in_Count: 1.2580 - std_absolute_Difference_in_Count: 0.9313 - MSE: 2.8257 - val_loss: 6.9806 - val_mean_Difference_in_Count: -1.4018 - val_std_Difference_in_Count: 2.1722 - val_mean_absolute_Difference_in_Count: 1.9036 - val_std_absolute_Difference_in_Count: 1.7575 - val_MSE: 6.7893
Epoch 3/100
936/936 [==============================] - 150s 160ms/

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,A1_X_test,Ac_Y_train,A1_Y_test,True)

Epoch 1/100
1464/1464 [==============================] - 235s 160ms/step - loss: 4.5041 - mean_Difference_in_Count: -0.1009 - std_Difference_in_Count: 1.4312 - mean_absolute_Difference_in_Count: 1.4080 - std_absolute_Difference_in_Count: 0.9934 - MSE: 4.0275 - val_loss: 3.9384 - val_mean_Difference_in_Count: -0.7333 - val_std_Difference_in_Count: 1.7876 - val_mean_absolute_Difference_in_Count: 1.5333 - val_std_absolute_Difference_in_Count: 1.1757 - val_MSE: 3.7333
Epoch 2/100
1464/1464 [==============================] - 234s 160ms/step - loss: 2.9405 - mean_Difference_in_Count: -0.0427 - std_Difference_in_Count: 1.3288 - mean_absolute_Difference_in_Count: 1.2542 - std_absolute_Difference_in_Count: 0.9233 - MSE: 2.8034 - val_loss: 3.8668 - val_mean_Difference_in_Count: 0.2333 - val_std_Difference_in_Count: 1.8382 - val_mean_absolute_Difference_in_Count: 1.5000 - val_std_absolute_Difference_in_Count: 1.0878 - val_MSE: 3.4333
Epoch 3/100
1464/1464 [==============================] - 233s 1

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,A2_X_test,Ac_Y_train,A2_Y_test,True)

Epoch 1/100
1464/1464 [==============================] - 235s 161ms/step - loss: 3.7164 - mean_Difference_in_Count: -0.0878 - std_Difference_in_Count: 1.2896 - mean_absolute_Difference_in_Count: 1.2665 - std_absolute_Difference_in_Count: 0.9229 - MSE: 3.2563 - val_loss: 4.6558 - val_mean_Difference_in_Count: -0.2969 - val_std_Difference_in_Count: 1.9937 - val_mean_absolute_Difference_in_Count: 1.6406 - val_std_absolute_Difference_in_Count: 1.1931 - val_MSE: 4.1406
Epoch 2/100
1464/1464 [==============================] - 233s 159ms/step - loss: 2.3043 - mean_Difference_in_Count: -0.0370 - std_Difference_in_Count: 1.1752 - mean_absolute_Difference_in_Count: 1.0960 - std_absolute_Difference_in_Count: 0.8214 - MSE: 2.1640 - val_loss: 4.2254 - val_mean_Difference_in_Count: -0.4375 - val_std_Difference_in_Count: 1.8655 - val_mean_absolute_Difference_in_Count: 1.6125 - val_std_absolute_Difference_in_Count: 1.1911 - val_MSE: 4.0250
Epoch 3/100
1464/1464 [==============================] - 233s 

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,A3_X_test,Ac_Y_train,A3_Y_test,True)

Epoch 1/100
1464/1464 [==============================] - 236s 161ms/step - loss: 8.4117 - mean_Difference_in_Count: -0.1442 - std_Difference_in_Count: 2.1158 - mean_absolute_Difference_in_Count: 2.1226 - std_absolute_Difference_in_Count: 1.4619 - MSE: 8.0732 - val_loss: 8.9014 - val_mean_Difference_in_Count: -0.8750 - val_std_Difference_in_Count: 2.8696 - val_mean_absolute_Difference_in_Count: 2.3750 - val_std_absolute_Difference_in_Count: 1.8329 - val_MSE: 9.0000
Epoch 2/100
1464/1464 [==============================] - 233s 159ms/step - loss: 5.4697 - mean_Difference_in_Count: -0.0894 - std_Difference_in_Count: 1.8766 - mean_absolute_Difference_in_Count: 1.7811 - std_absolute_Difference_in_Count: 1.2421 - MSE: 5.4498 - val_loss: 8.3984 - val_mean_Difference_in_Count: -0.0625 - val_std_Difference_in_Count: 2.9679 - val_mean_absolute_Difference_in_Count: 2.1875 - val_std_absolute_Difference_in_Count: 2.0068 - val_MSE: 8.8125
Epoch 3/100
1464/1464 [==============================] - 233s 

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,A4_X_test,Ac_Y_train,A4_Y_test,True)

Epoch 1/100
1464/1464 [==============================] - 237s 162ms/step - loss: 5.6737 - mean_Difference_in_Count: -0.1099 - std_Difference_in_Count: 1.6963 - mean_absolute_Difference_in_Count: 1.6804 - std_absolute_Difference_in_Count: 1.1691 - MSE: 5.1656 - val_loss: 4.3115 - val_mean_Difference_in_Count: -0.5411 - val_std_Difference_in_Count: 1.8467 - val_mean_absolute_Difference_in_Count: 1.4571 - val_std_absolute_Difference_in_Count: 1.2713 - val_MSE: 3.8768
Epoch 2/100
1464/1464 [==============================] - 235s 160ms/step - loss: 4.2257 - mean_Difference_in_Count: -0.0708 - std_Difference_in_Count: 1.6224 - mean_absolute_Difference_in_Count: 1.5476 - std_absolute_Difference_in_Count: 1.0916 - MSE: 4.0936 - val_loss: 5.2482 - val_mean_Difference_in_Count: 1.0473 - val_std_Difference_in_Count: 1.9660 - val_mean_absolute_Difference_in_Count: 1.7205 - val_std_absolute_Difference_in_Count: 1.4176 - val_MSE: 5.1312
Epoch 3/100
1464/1464 [==============================] - 235s 1

In [0]:
fitter_functions(create_Xception_model(),Ac_X_train,Ac_X_test,Ac_Y_train,Ac_Y_test,True)

Epoch 1/100
1464/1464 [==============================] - 236s 161ms/step - loss: 3.9974 - mean_Difference_in_Count: -0.1215 - std_Difference_in_Count: 1.2308 - mean_absolute_Difference_in_Count: 1.2435 - std_absolute_Difference_in_Count: 0.8852 - MSE: 3.6385 - val_loss: 5.5703 - val_mean_Difference_in_Count: 0.6156 - val_std_Difference_in_Count: 2.1232 - val_mean_absolute_Difference_in_Count: 1.7031 - val_std_absolute_Difference_in_Count: 1.4469 - val_MSE: 5.1125
Epoch 2/100
1464/1464 [==============================] - 234s 160ms/step - loss: 2.2807 - mean_Difference_in_Count: -0.0376 - std_Difference_in_Count: 1.1360 - mean_absolute_Difference_in_Count: 1.0824 - std_absolute_Difference_in_Count: 0.7990 - MSE: 2.1639 - val_loss: 4.4628 - val_mean_Difference_in_Count: 0.0492 - val_std_Difference_in_Count: 2.0300 - val_mean_absolute_Difference_in_Count: 1.5758 - val_std_absolute_Difference_in_Count: 1.3225 - val_MSE: 4.3680
Epoch 3/100
1464/1464 [==============================] - 235s 16

In [0]:
#Depicting the results
#%load_ext tensorboard
#%tensorboard --logdir logs

In [0]:
#Xception_conv_altered.load_weights('/content/drive/My Drive/model/eps=002_valLoss=16.4232.hdf5')

In [0]:
#idx = np.random.randint(0, len(Ac_X_test), 1)
#print(Xception_conv_altered.predict(Ac_X_test[idx]))
#print(Ac_Y_test[idx])